In [54]:
import numpy as np
import pandas as pd
import tempfile as tmp
import subprocess as subp
import re

from __future__ import division

## Constants 
## =================

In [55]:
GLUCOSE_GROWTH_CURVE = [[0,3,4,5,6,8,24,48,168,336],np.array([0.00267,0.01333,0.02567,0.06167,0.13567,0.19933,0.19500,0.17667,0.15233,0.14567])]
GLUCOSE_GROWTH_CURVE_STDDEV = np.array([0.00058,0.00208,0.00306,0.00058,0.01210,0.00321,0.00100,0.00306,0.00153,0.00208])
TIMES = [3,4,5,6,8,24,48,168,336]
# Constant mass shifts expected to be present at all residues of a specific type, e.g. carbamidomethylation of Cys from IAA
CONSTANT_MODS = {'C':57}


## Functions 
## =================

In [56]:
def parse_MODa(file,
               col_dtypes=[('DataSet',np.str),
                           ('Index' , np.int32),
                           ('ObservedMW', np.float64),
                           ('Charge'  , np.int32),
                           ('CalculatedMW' , np.float64),
                           ('DeltaMass' , np.float64),
                           ('Score' , np.int32),
                           ('Probability' , np.float64),
                          ('Peptide' , np.str),
                         ('giID' , np.int64),
                         ('PeptideStart' , np.int32),
                         ('PeptideEnd' , np.int32)]):
    """
    build a Dataframe from a one-peptide-per-row MODa file

    file: MODa file (best to concatenate all timepoint files together)
    col_dtypes: list of (ColumnName,Numpy datatype) pairs
    """
    
    # Fix some problematic text things 
    # NOTE THAT THESE MODIFY THE FILE IN-PLACE
    parse_temp = tmp.NamedTemporaryFile()
    p1 = subp.Popen(['perl','-pe','s/\~/\t/g',file],stdout=subp.PIPE)
    p2 = subp.Popen(['perl','-pe','s/\S+MURI_(\S+?)\.\S+/$1/'],stdin=p1.stdout,stdout=subp.PIPE)
    p3 = subp.Popen(['perl','-pe','s/gi\|//'],stdin=p2.stdout,stdout=subp.PIPE)
    
    # Filter out any header lines
    p4 = subp.Popen(['grep','-v','SpectrumFile'],stdin=p3.stdout,stdout=parse_temp)
    p4.communicate()
    df = pd.read_csv(parse_temp.name,sep="\t",header=None,names=zip(*col_dtypes)[0])#,dtype=dict(col_dtypes))#,error_bad_lines=False,keep_default_na=False,verbose=True)
    parse_temp.close()
    return df

def get_mod(peptide,constants=CONSTANT_MODS):
    """
    RegEx out the modified residue & location from a MODa peptide field

    peptide: peptide seq w/ MODa annotation (string)
    """
    hit = re.search('\w?\.(\w*)((\+|-)\d+)(\w*)\.\w?',peptide)
    full_pept = ""
    modification = 0
    modloc = -1
    modres = '-'
    if hit:
        full_pept=hit.group(1) + hit.group(4)
        modification = int(hit.group(2))
        modloc=len(hit.group(1)) - 1
        modres=hit.group(1)[-1]
        if modres in constants.keys():
            modification += constants[modres]
    else:
        pept_seq_search = re.search('\w?\.(\w+)\.\w?',peptide)
        full_pept=pept_seq_search.group(1)
        modification = 0
    return (full_pept,modification,modloc,modres)


## Parse MODa File into a pandas DataFrame
## =================

In [57]:
print all_pept_raw.shape
print all_pept.shape

(2527135, 12)
(2527135, 28)


In [59]:
# parse the concatenated MODa file
all_pept_raw = parse_MODa("data/MURI_ALL.id.txt")
print all_pept_raw.head()

# Load protein table and giID list
proteinTable = pd.read_csv("data/REL606_ProteinTable167_161539.txt",sep="\t")
gi_ids = pd.read_csv("data/gi_to_name_list_shrt.txt",sep="\t",header=None,names=['ProteinProduct','LocusTag','giID'])

# merge to build giID table
proteins = pd.merge(proteinTable,gi_ids,on='LocusTag')
print "proteins len: %d" % (len(proteins),)

# Build table relating DataSet ID to time, biol. replicate, and tech. replicate
tp = set(all_pept_raw['DataSet'])
srt_tp = sorted(tp, cmp=(lambda x,y: cmp((int(x[:-1]),x[-1]), (int(y[:-1]),y[-1]))))
times = [3,3,4,4,5,5,6,6,8,8,24,24,48,48,168,168,336,336] * 3
breplicates = [1] * 18
breplicates.extend([2] * 18)
breplicates.extend([3] * 18)
treplicates = [1,2] * 27
time_ID = pd.DataFrame(data=zip(times,breplicates,treplicates,srt_tp), columns=['Time','BiolRep','TechRep','DataSet'])

# merge (join) to match giIDs to human-readable gene info
all_pept = pd.merge(all_pept_raw,proteins,on="giID")
# join all_pept to time_ID to match AG3C dataset ID to timepoint and replicate
all_pept = pd.merge(all_pept,time_ID,on="DataSet")

# Extract modification info from MODa peptide sequence string
pept_seqs = all_pept['Peptide']
pept_seqs,mods,modloc,modres = zip(*pept_seqs.apply(get_mod))

#Add modification info columns to all_pept
all_pept["PeptideSeq"] = pd.Series(pept_seqs)
all_pept["MassShift"] = pd.Series(mods)
all_pept["ModifiedPosition"] = pd.Series(modloc)
all_pept["ModifiedResidue"] = pd.Series(modres)

# Drop columns if not needed
all_pept = all_pept.drop(["RepliconName","RepliconAccession","ProteinProduct_y","COGs"],axis=1)

# Check! [replace w/ an assertion if do a lot more of these]
print all_pept.head().to_string()

  DataSet  Index  ObservedMW  Charge  CalculatedMW  DeltaMass  Score  \
0    100a    128    986.5384       2      986.5396    -0.0013     23   
1    100a    153   1958.0489       4     1958.0091     0.0398     26   
2    100a    157    923.7056       3      923.5498     0.1558     12   
3    100a    164    986.5396       2      986.5396    -0.0000     21   
4    100a    167    986.5377       2      986.5396    -0.0019     22   

   Probability                    Peptide       giID  PeptideStart  PeptideEnd  
0       0.3035              K.LAGDLETLR.S  254160296           219         227  
1       0.1372  R.FKDP+55DAALPEIALYVGGK.L  254161376           270         287  
2       0.1366           R.EAALTLL-19GR.A  254163538           322         330  
3       0.1744              K.LAGDLETLR.S  254160296           219         227  
4       0.1571              K.LAGDLETLR.S  254160296           219         227  
proteins len: 4201
  DataSet  Index  ObservedMW  Charge  CalculatedMW  DeltaMass 

In [ ]:
# Output raw peptide sequences for external MW calculation
all_pept.PeptideSeq.to_csv("data/all_pept_PeptideSeq.txt",index=False)

In [ ]:
# Use ExPaSY protein tools peptide MW calculator to generate table of monoisotopic molecular weights for the unmodified peptides
#
# Output formats may vary if you use something else...

mono_MW = pd.read_csv("data/all_pept_monoMW.txt",header=None,delim_whitespace=True,names=["PeptideSeq","pI","CalculatedUnmodMW"])
mono_MW.drop(['pI'],axis=1,inplace=True)
all_pept_calcMW = pd.merge(all_pept,mono_MW,on="PeptideSeq",how='left')

In [ ]:
all_pept_calcMW.to_csv("data/all_pept.tsv",sep="\t")